In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
from data_convertort.convert_to_csv import TxtToCSVConverter
from ab_testing.data_processor import DataProcessor
from ab_testing.hypothesis_tester import HypothesisTester
from ab_testing.report_generator import ReportGenerator
from data_cleaner.data_cleaner import DataCleaner

In [2]:
converter = TxtToCSVConverter('../data/MachineLearningRating_v3.txt', '../data/MachineLearningRating_v3.csv')
converter.convert()

Output file already exists: ../data/MachineLearningRating_v3.csv. Conversion skipped.


In [3]:

df =pd.read_csv('../data/MachineLearningRating_v3.csv')


C:\Users\sefuwan.feysa\AppData\Local\Temp\ipykernel_28508\3417818192.py:1: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df =pd.read_csv('../data/MachineLearningRating_v3.csv')


In [9]:
# Clean the data
cleaner = DataCleaner(df)
cleaner.clean_gender()
cleaner.add_margin_column()

# Get the cleaned data and display only unique Title and Gender pairs
cleaned_df = cleaner.get_cleaned_data()

# Display the unique values
cleaned_df[['Title', 'Gender']].drop_duplicates().head(20)


,Title,Gender
0,Mr,1
478,Dr,1
1034,Miss,0
699059,Mrs,0
749286,Ms,0


In [10]:

# Instantiate classes
data_processor = DataProcessor(cleaned_df)
hypothesis_tester = HypothesisTester()


In [11]:
# Null Hypothesis 1: Risk difference across provinces
group_a_prov, group_b_prov = data_processor.segment_data(
    'Province', 
    cleaned_df['Province'] == 'Gauteng', 
    cleaned_df['Province'] == 'Western Cape'
)
p_value_prov = hypothesis_tester.t_test(group_a_prov, group_b_prov, 'TotalClaims')
result_prov = hypothesis_tester.analyze_results(p_value_prov)
hypothesis_tester.add_result('Risk difference across provinces', p_value_prov, result_prov)


In [12]:
# Null Hypothesis 2: Risk difference between zip codes
group_a_zip, group_b_zip = data_processor.segment_data(
    'PostalCode', 
    cleaned_df['PostalCode'] == 1000, 
    cleaned_df['PostalCode'] == 2000
)
p_value_zip = hypothesis_tester.t_test(group_a_zip, group_b_zip, 'TotalClaims')
result_zip = hypothesis_tester.analyze_results(p_value_zip)
hypothesis_tester.add_result('Risk difference across zip codes', p_value_zip, result_zip)

In [13]:
# Null Hypothesis 3: Margin difference between zip codes
group_a_margin, group_b_margin = data_processor.segment_data(
    'PostalCode', 
    cleaned_df['PostalCode'] == 1000, 
    cleaned_df['PostalCode'] == 2000
)
# Assuming we have a 'Margin' column: Margin = TotalPremium - TotalClaims
# df['Margin'] = df['TotalPremium'] - df['TotalClaims']
p_value_margin = hypothesis_tester.t_test(group_a_margin, group_b_margin, 'Margin')
result_margin = hypothesis_tester.analyze_results(p_value_margin)
hypothesis_tester.add_result('Margin difference between zip codes', p_value_margin, result_margin)

In [14]:
# Null Hypothesis 4: Risk difference between men and women
group_a_gender, group_b_gender = data_processor.segment_data(
    'Gender', 
    cleaned_df['Gender'] == 1, 
    cleaned_df['Gender'] == 0
)
p_value_gender = hypothesis_tester.t_test(group_a_gender, group_b_gender, 'TotalClaims')
result_gender = hypothesis_tester.analyze_results(p_value_gender)
hypothesis_tester.add_result('Risk difference between men and women', p_value_gender, result_gender)


In [15]:
# Generate report
report_generator = ReportGenerator(hypothesis_tester.results)
print(report_generator.generate_report())


A/B Hypothesis Testing Report
Hypothesis: Risk difference across provinces
P-Value: 0.0621523145227998
Result: Fail to reject the null hypothesis
----------------------------------------
Hypothesis: Risk difference across zip codes
P-Value: 1.0871786315077791e-36
Result: Reject the null hypothesis
----------------------------------------
Hypothesis: Margin difference between zip codes
P-Value: 7.662076097924192e-12
Result: Reject the null hypothesis
----------------------------------------
Hypothesis: Risk difference between men and women
P-Value: 0.020783336597980866
Result: Reject the null hypothesis
----------------------------------------

